In [ ]:
# Install the required libraries
!pip install transformers datasets


In [2]:
# Import necessary libraries
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

In [3]:
# Load the pre-trained T5 model and tokenizer fine-tuned on WikiSQL
model_name = "mrm8488/t5-small-finetuned-wikisql"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
# Load the WikiSQL dataset
dataset = load_dataset("wikisql")

The repository for wikisql contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikisql.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [5]:

def convert_nl_to_sql(question):
    # Preprocess the input question
    input_text = f"translate English to SQL: {question}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate the SQL query
    outputs = model.generate(input_ids, max_new_tokens=100)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return sql_query

In [6]:
# Example questions from the WikiSQL dataset
example_questions = [
    "What is the name of the country with the highest population?",
    "How many people live in India?",
    "List all countries with a population greater than 100 million."
]

# Convert each example question to SQL and print the result
for question in example_questions:
    sql_query = convert_nl_to_sql(question)
    print(f"Question: {question}")
    print(f"SQL Query: {sql_query}\n")


Question: What is the name of the country with the highest population?
SQL Query: SELECT Name FROM table WHERE Population = highest

Question: How many people live in India?
SQL Query: SELECT COUNT Live FROM table WHERE Country = india

Question: List all countries with a population greater than 100 million.
SQL Query: SELECT Countries FROM table WHERE Population > 100 million

